# KNIT6 OWL Method1 (Docker containers)

This Notebook shows how to run OWL simply by runnign OWL Docker containers without the help of meas_node

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from mflib import owl


fablib = fablib_manager()

## Identify a slice

In [ ]:
slice_name= "KNIT6_plain_slice"

try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")   
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
nodes = slice.get_nodes()
node1 = slice.get_node("Node1")
node2 = slice.get_node("Node2")

## Set up OWL

Args

- `local_owl_dir`: Directory where owl output will be downloaded
- `remote_out_dirr`: Directory on exp node where `*.pcap` files will be saved during experiments
- `image_name (default="fabrictestbed/owl:0.1.3")`: Docker OWL image to pull
- `remote_conf_dir (default=None)`: Directory on exp node where `owl.conf` and `links.json` are kept. To use those features, refer to the documetation.

In [ ]:
local_owl=f'/home/fabric/work/owl_output/{slice_name}'
remote_output='/home/rocky/owl-output' 

In [ ]:
owl = owl.OwlDocker(local_owl_dir=local_owl, remote_out_dir=remote_output)

## Look up IPs of nodes and decide which IP address to use 

In [ ]:
for node in nodes:
    print(f"{node.get_name()}: {owl.list_experiment_ip_addrs(node)} at {node.get_site()}")

In [ ]:
node1_ip = owl.list_experiment_ip_addrs(node1)[0]
node2_ip = owl.list_experiment_ip_addrs(node2)[0]

print(node1_ip, node2_ip)

## (Optional) Check output files from previous runs

In [ ]:
for node in nodes:
    print(f"\n{node.get_name()}:")
    owl.list_remote_output(node)

### Delete if necessary

In [ ]:
for node in nodes:
    owl.delete_remote_output(node)

## Set up OWL via Docker Containers

### (Optional) Confirm the node has PTP enalbed and Docker daemon running

In [ ]:
for node in nodes:
    owl.check_node_environment(node)

### Create Necessary directories on EXP nodes

In [ ]:
# This will be owl.create_remote_out_dir(node)
for node in nodes:
    owl.create_remote_out_dir(node)

## Pull Docker image

In [ ]:
for node in nodes:
    owl.pull_owl_docker_image(node)

# Start Sender on Node1!

This starts an OWL container and run the sender code on the node.

Args:
- `src_node`
- `dst_ip`: destination IPv4 address (str)
- `name (default="fabric-owl')`: container name
- `frequency (default=0.5)`: interval at which probe packets will be sent
- `seq_n (default=1234)`: number at which sequence number should start
- `duration (default=180)`: how long (sec) to run OWL
- `python_time (defualt=False)`: set to True if prefer to call Python time.time_ns() for timestamping rather than the MF-OWL timestamp code

In [ ]:
owl.start_owl_sender(node1, node2_ip)

# Start Capturer on Node2!

Likewise, this starts an OWL container and run the capturer code on the destination node 

Args:
- `dst_node`: node on which this should run
- `receiving_ip`: destination IPv4 address to be used (str)
- `name (default="fabric-owl')`: container name
- `pcap_time_limit (default=360)`: interval (sec) at which tcpdump starts a new pcap file
- `duration (default=180)`: how long (sec) to run OWL

In [ ]:
owl.start_owl_receiver(node2, node2_ip)

### (Optional) Check the container log while OWL is running

In [ ]:
for node in nodes:
    owl.get_owl_log(node)

# Stop Docker containers on both Node1 and Node2

In [ ]:
for node in nodes:
    owl.stop_owl_docker(node)

# Download output

## (Optional) Check if pcap files have been generated

In [ ]:
for node in nodes:
    print(f"\n****** {node.get_name()} ******")
    owl.list_remote_output(node)

## download

In [ ]:
for node in nodes:
    owl.download_output(node)

## ⚠️☠️ Run ONLY IF you want to remove OWL (containers and images)

In [ ]:
# for node in nodes:
#     owl.remove_owl_docker_image(node)